<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Import-scraped-data" data-toc-modified-id="Import-scraped-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import scraped data</a></span></li><li><span><a href="#Data-Set-Features" data-toc-modified-id="Data-Set-Features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Set Features</a></span></li><li><span><a href="#Things-To-Do" data-toc-modified-id="Things-To-Do-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Things To Do</a></span><ul class="toc-item"><li><span><a href="#Columns:-renaming-and-reordering" data-toc-modified-id="Columns:-renaming-and-reordering-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Columns: renaming and reordering</a></span></li><li><span><a href="#Drop-&quot;with-milk&quot;-column" data-toc-modified-id="Drop-&quot;with-milk&quot;-column-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Drop "with milk" column</a></span></li><li><span><a href="#Duplicates" data-toc-modified-id="Duplicates-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Duplicates</a></span></li><li><span><a href="#Fix-Acidity/Structure-and-acidity-scores" data-toc-modified-id="Fix-Acidity/Structure-and-acidity-scores-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Fix Acidity/Structure and acidity scores</a></span></li><li><span><a href="#Fix-price-column" data-toc-modified-id="Fix-price-column-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Fix price column</a></span></li><li><span><a href="#Split-Agtron-score-into-whole-bean-and-ground" data-toc-modified-id="Split-Agtron-score-into-whole-bean-and-ground-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Split Agtron score into whole bean and ground</a></span></li><li><span><a href="#Countries" data-toc-modified-id="Countries-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Countries</a></span></li></ul></li><li><span><a href="#Fix-data-types" data-toc-modified-id="Fix-data-types-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fix data types</a></span></li><li><span><a href="#Code-Holding" data-toc-modified-id="Code-Holding-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Code Holding</a></span></li></ul></div>

# Import Libraries

In [910]:
import numpy as np 
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pylab import rcParams

plt.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import itertools
import statsmodels.api as sm
import operator

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

import time

import re
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import Request, urlopen

# Import scraped data

In [2]:
df = pd.read_csv('AN_coffee_reviews.csv')

In [3]:
df.head()

,Unnamed: 0,Acidity,Acidity /Structure,Aftertaste,Agtron,Aroma,Body,Coffee Origin,Est. Price,Flavor,Review Date,Roast Level,Roaster Location,With Milk,bottom_line,rating,roaster,title
0,0,NaN,9.0,8.0,58/76,9,9,"Tarrazu, Costa Rica",$23.00/12 ounces,9,April 2020,Medium-Light,"Billings, Montana",NaN,Enjoying this Costa Rica evokes the pleasures ...,94,Revel Coffee,Costa Rica Luis Campos Anaerobic
1,1,NaN,9.0,8.0,60/74,9,9,"Valle del Cauca growing region, Colombia",$23.50/12 ounces,9,April 2020,Medium-Light,"Topeka, Kansas",NaN,A bombshell of a coffee with enough personalit...,94,PT's Coffee Roasting Co.,Colombia Granja La Esperanza Tres Dragones
2,2,NaN,8.0,8.0,50/70,9,8,"Nyeri growing region, south-central Kenya",NT $530/200 grams,9,April 2020,Medium,"Taipei, Taiwan",NaN,A developed medium roast emphasizes this Kenya...,92,Quartet Kaffe,Kenya Thageini
3,3,NaN,9.0,8.0,58/76,9,9,"Huila, Colombia",$16.50/12 ounces,9,March 2020,Medium-Light,"Jackson, Mississippi",NaN,An exceptional Colombia cup with sweet and tar...,94,BeanFruit Coffee Co.,Colombia Finca La Loma Microlot
4,4,NaN,NaN,7.0,56/67,7,8,Sumatra,$12.36/12 ounces,8,March 2020,Medium-Light,"Telluride, Colorado",8,"The sweet, fruit-on-the-edge of decay characte...",88,Telluride Coffee Roasters,Aged Sumatra Espresso


# Data Set Features 

The explanation of the features was taken from the CoffeeReviews website (https://www.coffeereview.com/interpret-coffee/).  A rating system of 1 (low) to 10 (high) is used to assess coffee beans.

- Acidity or Acidity/Structure: Acidity is the bright, dry sensation that enlivens the taste of coffee. Without acidity coffee is dull and lifeless. Acidity is not a sour sensation, which is a taste defect, nor should it be excessively drying or astringent, though it sometimes is. At best it is a sweetly tart vibrancy that lifts the coffee and pleasurably stretches its range and dimension. Acidity can be delicate and crisp, lush and rich, powerfully tart but sweet, or backgrounded but vibrant, to cite only a few positive ways to characterize it. The darker a coffee is roasted, the less overt acidity it will display.

- Flavor and Aftertaste: Flavor and aftertaste include everything not suitably described under the categories aroma, acidity and body. An assessment of flavor includes consideration of the balance of basic tastes – sweet, bitter and sour in particular, and specific aroma and flavor notes, which are many and can be described by associations like floral (honeysuckle, rose, lilac, etc.), nuances of sweetness (honey, molasses, brown sugar), aromatic wood (cedar, pine, sandalwood) and above all fruit (from bright citrus to lusher, rounder fruit like apricot or plum, or pungent fruit like black currant or mango). Descriptors of flavor may also be global – balanced, deep, delicate, etc. Aftertaste or finish describes reflects sensations that linger after the coffee has been swallowed (or spit out). Generally we tend to reward coffees in which pleasing flavor notes continue to saturate the aftertaste long after the coffee is gone, and the sensations left behind are generally sweet-toned rather than excessively bitter or drying and astringent.

- Agtron: An agtron maching reflects light on a sample of coffee to objectively assign a number to the bean's roast color.  The smaller the number, the darker the roast. It is a precise measure of the degree of roast. We (CoffeeReviews.com) use the M-Basic or “Gourmet” Agtron scale, and for each coffee reviewed we present readings both of the whole beans before grinding (the number preceding the slash) and the same beans after grinding (the number after the slash). For example, a reading of 55/68 would describe a coffee with an external, whole-bean M-Basic reading of 55, and a ground reading of 68. 

- Aroma: How intense and pleasurable is the aroma when the nose first descends over the cup and is enveloped by fragrance? Aroma also provides a subtle introduction to various nuances of acidity, taste and flavor: bitter and sweet tones, fruit, flower or herbal notes, and the like.

- Body: Body and mouthfeel describe sensations of weight and texture. Body can be light and delicate, heavy and resonant, thin and disappointing; in texture it can be silky, plush, syrupy, lean or thin.

- Coffee Origin: The source of green coffee beans.

- Est. Price: Price to the consumer.

- Review Date: Date of reveiw by CoffeeReviews.com

- Roast Level: See Agtron score.  The degree of roast level affects a coffee's flavor profile. Based on the Agtron readings, a general descriptive terms for roast color is provided – light, medium, medium-dark, dark, etc. – for each coffee reviewed based on terminology developed by the Specialty Coffee Association of America.

- Roaster Location: Location of roaster of green coffee beans.

- With Milk: No explanation given and few coffees have this score

- Bottom Line: Summary of review

- Rating: The scale for the overall coffee ratings runs from 50 to 100, and reflects the reviewers’ overall subjective assessment of a coffee’s sensory profile as manifest in the five categories aroma, acidity, body and flavor and aftertaste. 

- Roaster: Name of company that roasts the beans

- Title: Name of the coffee

# Things To Do

In [ ]:
# DONE: redo proposal, posted on Trello

# DONE, NOT MANY AVAILABLE: look at Kaggle dataset kernals for viz ideas

# EDA
#  1. NOT NEEDED: set index
#  2. DONE: fix columns (reorder)
#  3. fix data types 
#  4. DONE: fix pricing (first set null values to 0, graph time v price? before deleting null)
#  5. DONE: fix and check for missing data / nulls
#  6. DONE: potentially change some column names
#  7. DONE: describe column names (agtron, aftertase)
#  8. DONE: fix coffee origin to just a country and find altitude?
#  9. DONE: separate agtron scores

# visualizations
#  1. Map of world with coffee origin plotted, w layers
#  https://gis.stackexchange.com/questions/198530/plotting-us-cities-on-a-map-with-matplotlib-and-basemap
#  2. price v rating
#  3. rating v agtron or agtron v origin
#  4. confusion matrix
#  5. Distribution of ratings
#  6. Feature importance

# What is the so-what?  What about your findings is actionable?
# Supervised learning
# Sentiment analysis
# A well-defined question with a well-defined answer

## Columns: renaming and reordering

In [4]:
df.columns

Index(['Unnamed: 0', 'Acidity ', 'Acidity /Structure ', 'Aftertaste', 'Agtron',
       'Aroma', 'Body', 'Coffee Origin', 'Est. Price', 'Flavor', 'Review Date',
       'Roast Level', 'Roaster Location', 'With Milk', 'bottom_line', 'rating',
       'roaster', 'title'],
      dtype='object')

In [5]:
# Re-order columns

df = df[['title','roaster','Roaster Location','Coffee Origin','rating','Roast Level','Review Date','Est. Price',
         'Agtron','Flavor','Body','Aroma','Aftertaste','Acidity /Structure ', 'Acidity ','With Milk','bottom_line']]

In [6]:
df.head()

,title,roaster,Roaster Location,Coffee Origin,rating,Roast Level,Review Date,Est. Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity /Structure,Acidity,With Milk,bottom_line
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,NaN,NaN,Enjoying this Costa Rica evokes the pleasures ...
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,NaN,NaN,A bombshell of a coffee with enough personalit...
2,Kenya Thageini,Quartet Kaffe,"Taipei, Taiwan","Nyeri growing region, south-central Kenya",92,Medium,April 2020,NT $530/200 grams,50/70,9,8,9,8.0,8.0,NaN,NaN,A developed medium roast emphasizes this Kenya...
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,NaN,NaN,An exceptional Colombia cup with sweet and tar...
4,Aged Sumatra Espresso,Telluride Coffee Roasters,"Telluride, Colorado",Sumatra,88,Medium-Light,March 2020,$12.36/12 ounces,56/67,8,8,7,7.0,NaN,NaN,8,"The sweet, fruit-on-the-edge of decay characte..."


In [7]:
# Rename columns

df.rename(columns={'title':'Coffee Name'}, inplace=True)
df.rename(columns={'roaster':'Roaster Name'}, inplace=True)
df.rename(columns={'rating':'Rating'}, inplace=True)
df.rename(columns={'Est. Price':'Price'}, inplace=True)
df.rename(columns={'bottom_line':'Bottom Line'}, inplace=True)

## Drop "with milk" column

In [8]:
# Drop 'with milk' column since there are so many null values

df = df.drop(['With Milk'], axis=1)

In [9]:
df.head()

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity /Structure,Acidity,Bottom Line
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,NaN,Enjoying this Costa Rica evokes the pleasures ...
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,NaN,A bombshell of a coffee with enough personalit...
2,Kenya Thageini,Quartet Kaffe,"Taipei, Taiwan","Nyeri growing region, south-central Kenya",92,Medium,April 2020,NT $530/200 grams,50/70,9,8,9,8.0,8.0,NaN,A developed medium roast emphasizes this Kenya...
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,NaN,An exceptional Colombia cup with sweet and tar...
4,Aged Sumatra Espresso,Telluride Coffee Roasters,"Telluride, Colorado",Sumatra,88,Medium-Light,March 2020,$12.36/12 ounces,56/67,8,8,7,7.0,NaN,NaN,"The sweet, fruit-on-the-edge of decay characte..."


## Duplicates

In [10]:
# Find and deal with duplicate reviews

print("There are {} duplicated values in this dataset.".format(df.duplicated().sum()))
duplicates = df[df.duplicated(keep=False)]
duplicates.sort_values(by=['Coffee Name']).head()
df.drop_duplicates(inplace=True)

There are 3 duplicated values in this dataset.


In [11]:
print("There are {} duplicated values in this dataset.".format(df.duplicated().sum()))

There are 0 duplicated values in this dataset.


In [ ]:
# Re set index, if so desire

# df.set_index("ID", inplace=True)
# df.set_index('Coffee Name', inplace=True)

In [12]:
df.dtypes

Coffee Name             object
Roaster Name            object
Roaster Location        object
Coffee Origin           object
Rating                  object
Roast Level             object
Review Date             object
Price                   object
Agtron                  object
Flavor                  object
Body                    object
Aroma                   object
Aftertaste             float64
Acidity /Structure     float64
Acidity                 object
Bottom Line             object
dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5512 entries, 0 to 5514
Data columns (total 16 columns):
Coffee Name            5512 non-null object
Roaster Name           5512 non-null object
Roaster Location       5510 non-null object
Coffee Origin          4932 non-null object
Rating                 5512 non-null object
Roast Level            5082 non-null object
Review Date            5512 non-null object
Price                  3403 non-null object
Agtron                 5512 non-null object
Flavor                 5493 non-null object
Body                   5498 non-null object
Aroma                  5472 non-null object
Aftertaste             4555 non-null float64
Acidity /Structure     765 non-null float64
Acidity                3797 non-null object
Bottom Line            5385 non-null object
dtypes: float64(2), object(14)
memory usage: 732.1+ KB


## Fix Acidity/Structure and acidity scores

In [228]:

# df['Acidity /Structure '].astype(str).astype(float)

In [14]:
df['Acidity '] = df['Acidity '].apply(pd.to_numeric, errors='coerce')
# df['Acidity '].astype(str).astype(float)

In [15]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5512 entries, 0 to 5514
Data columns (total 16 columns):
Coffee Name            5512 non-null object
Roaster Name           5512 non-null object
Roaster Location       5510 non-null object
Coffee Origin          4932 non-null object
Rating                 5512 non-null object
Roast Level            5082 non-null object
Review Date            5512 non-null object
Price                  3403 non-null object
Agtron                 5512 non-null object
Flavor                 5493 non-null object
Body                   5498 non-null object
Aroma                  5472 non-null object
Aftertaste             4555 non-null float64
Acidity /Structure     765 non-null float64
Acidity                3775 non-null float64
Bottom Line            5385 non-null object
dtypes: float64(3), object(13)
memory usage: 732.1+ KB


In [16]:
# checking for null values in a/s and a
mask = df['Acidity /Structure '].isnull().sum() & df['Acidity '].isnull().sum()

# try to fill 
# df['Acidity '].fillna(df['Acidity /Structure '])
# df.info()

# df['Acidity'] = ['yes' if x >= 50 else 'no' for x in df['salary']]

# df['Acidity'] = df.loc[df['Acidity '] > 0 df = df.apply(pd.to_numeric, errors='coerce')]

In [17]:
# print (df[mask].info())
print(mask.sum())

649


In [18]:
# Create a new column to combine a/s and a, delete null values
# The new column name does not have spaces in the string

# frame[['b','c']].apply(lambda x: x['c'] if x['c']>0 else x['b'], axis=1)
# https://stackoverflow.com/questions/37443082/using-lambda-if-condition-on-different-columns-in-pandas-dataframe

df['Acidity'] = df[['Acidity /Structure ','Acidity ']].apply(lambda x: x['Acidity '] if x['Acidity '] > 0 else x['Acidity /Structure '], axis=1)



In [19]:
df = df.drop(['Acidity /Structure '], axis=1)
df = df.drop(['Acidity '], axis=1)

In [20]:
df.head(3)

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Bottom Line,Acidity
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,Enjoying this Costa Rica evokes the pleasures ...,9.0
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,A bombshell of a coffee with enough personalit...,9.0
2,Kenya Thageini,Quartet Kaffe,"Taipei, Taiwan","Nyeri growing region, south-central Kenya",92,Medium,April 2020,NT $530/200 grams,50/70,9,8,9,8.0,A developed medium roast emphasizes this Kenya...,8.0


In [21]:
df.columns

Index(['Coffee Name', 'Roaster Name', 'Roaster Location', 'Coffee Origin',
       'Rating', 'Roast Level', 'Review Date', 'Price', 'Agtron', 'Flavor',
       'Body', 'Aroma', 'Aftertaste', 'Bottom Line', 'Acidity'],
      dtype='object')

In [22]:
# Re-order columns again

df = df[['Coffee Name','Roaster Name','Roaster Location','Coffee Origin','Rating','Roast Level','Review Date','Price',
         'Agtron','Flavor','Body','Aroma','Aftertaste', 'Acidity','Bottom Line']]

In [ ]:
# copy of dataframe

# df_copy = df.copy()

In [23]:
# Drop rows with null values

df = df.dropna(axis=0, subset=['Price','Roaster Location', 'Roast Level','Flavor','Aroma','Aftertaste','Acidity'])
# df = df.dropna(axis=0, subset=['Price'])

## Fix price column

In [88]:
# Fix price column

df.Price.value_counts()

$18.00/12 ounces        104
$15.00/12 ounces         64
$16.00/12 ounces         58
$13.99/12 ounces         57
$17.00/12 ounces         54
$14.50/12 ounces         41
$14.95/12 ounces         41
$19.00/12 ounces         39
$16.50/12 ounces         38
$20.00/12 ounces         37
$14.99/12 ounces         35
$17.50/12 ounces         33
$16.95/12 ounces         31
$14.00/12 ounces         31
$15.50/12 ounces         26
$12.99/12 ounces         26
$15.75/12 ounces         25
$17.95/12 ounces         25
$19.50/12 ounces         24
$13.95/12 ounces         24
$15.95/12 ounces         24
$13.50/12 ounces         24
$21.00/12 ounces         23
$12.00/12 ounces         22
$19.95/12 ounces         22
$18.95/12 ounces         18
$22.00/12 ounces         17
$18.50/12 ounces         17
$17.99/12 ounces         16
$12.95/12 ounces         14
                       ... 
$39.95/16 ounces          1
$16.22/12 ounces          1
CNY $90/227 grams         1
AED $95.00/250 grams      1
$25.95/12 ounces    

In [188]:
# Create a series of price values
# Series is a one-dimensional labeled array capable of holding data of any type 
# (integer, string, float, python objects, etc.). The axis labels are collectively called index

prices = df.Price
# prices[0:10]

182    $16.25/10 ounces
184     $60.00/8 ounces
185     $54.95/6 ounces
187     $60.00/8 ounces
188     $40.00/4 ounces
191     $50.00/8 ounces
194    $19.00/12 ounces
198    $17.10/12 ounces
200    $12.00/12 ounces
201    $12.00/12 ounces
202    $18.70/12 ounces
204     $39.95/4 ounces
206    $15.45/12 ounces
210    $22.00/12 ounces
211     $19.65/8 ounces
212    $16.20/12 ounces
213    $34.99/16 ounces
215    $25.00/12 ounces
216    $45.00/16 ounces
220    $21.00/12 ounces
223    $23.00/12 ounces
224    $19.00/12 ounces
225     $25.75/4 ounces
226    $18.00/12 ounces
227    $23.00/12 ounces
228    $17.00/12 ounces
229    $19.00/12 ounces
230    $60.00/10 ounces
235     $35.00/8 ounces
236     $49.95/4 ounces
             ...       
311    $19.00/12 ounces
313    $16.20/12 ounces
315    $13.99/12 ounces
317    $15.95/12 ounces
318     $37.50/8 ounces
319    $23.00/12 ounces
320    $15.95/12 ounces
321    $18.00/12 ounces
323     $19.00/8 ounces
325    $19.95/12 ounces
327    $16.95/12

In [182]:
# last item of series

prices.iloc[-1]

'$11.95/16 ounces'

In [100]:
currencies = []
bad_indexes = []
dollars = 0
non_dollars = 0
for i, price in prices.items():
    price_split = price.split('$')
    if price_split[0] == '':
        dollars += 1
    else:
        non_dollars += 1
        bad_indexes.append(i)
    if price_split[0].strip() not in currencies:
        currencies.append(price_split[0].strip())
print(len(prices.keys()))
print("Dollars: {}".format(dollars))
print("Other: {}".format(non_dollars))
print(currencies)
print(bad_indexes)

2773
Dollars: 2382
Other: 391
['', 'NT', 'IDR', 'CAD', 'AUD', 'See website for more information', 'AED', '18.00/12 ounces', 'NTD', '#23.00/12 ounces', 'KRW', 'NT 400/227 grams', 'HKD', 'KRW 19,000/200 grams', 'RMB', 'CNY', 'KRW 40,000/500 grams', 'KRW 32,000/500 grams', 'See review note', 'THB', 'TWD', 'US', 'RMB 235/125 g', 'HK', 'KRW 50,000/200 grams', 'KRW 95,000/100 grams', '£5.50/250 grams', '£7.00/250 grams', '£6.50/250 grams', '£5.00/227 grams', '£6.50 for 250g', '£7.45/250 grams', 'KRW 35,000/500 grams', 'KRW 15,000/100 grams', 'E 50.00/250 grams', 'KRW 10,000/100 grams', 'KRW 35,000/100 grams', 'GTQ 100/12 ounces', 'Not disclosed.', 'Not Available', 'Price:', 'THB 150/250 g.', 'THB 120/ 250 g.']
[2, 10, 43, 61, 63, 64, 73, 74, 76, 82, 83, 84, 85, 86, 87, 88, 98, 102, 106, 109, 112, 116, 120, 121, 122, 130, 131, 143, 147, 149, 150, 153, 158, 162, 171, 176, 178, 183, 186, 189, 190, 192, 193, 205, 207, 208, 209, 214, 218, 219, 221, 259, 271, 273, 279, 303, 304, 314, 322, 326, 329

In [104]:
df.drop(bad_indexes, inplace=True)


In [105]:
df.head()

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...


In [117]:
# remove indexes without oz or ounces

units = []
ounces = 0
bad_units = []
for i, price in df.Price.items():
    unit = ''.join(price.split()[1:])
    if 'oz' in unit or 'ounces' in unit:
        ounces += 1
    else:
        bad_units.append(i)
    if price.split()[1:] not in units:
        units.append(price.split()[1:])
print(units)
print(ounces)
print(bad_units)

[['ounces'], ['ounces', '(includes', 'shipping)'], ['ounces;', '$50.00/5', 'pounds'], ['grams'], ['7', 'ounces'], ['ounces;', 'limited', 'availability'], ['ounces', '(packaged', 'as', 'a', '"duo"', 'with', 'Bourbon', 'Rey', 'Guatemala)'], ['ounces', '(packaged', 'as', 'a', '"duo"', 'with', 'the', 'Bourbon', 'Rey', 'Jamaica)'], ['ounces*'], ['700/227', 'grams'], ['pounds'], ['ounces', 'online;', '$12.99/ounces', 'in', 'stores'], ['capsules'], ['K-cups'], ['NTD/16', 'ounces'], ['K-Cups'], ['ounces', '(free', 'shipping)'], [], ['ounces', '(see', 'notes)'], ['NTD/100', 'grams'], ['/', '12', 'ounces'], ['/', '8', 'ounces'], ['single-serve', 'capsules'], ['RealCup', 'capsules'], ['Vue-Packs'], ['single-serve', 'pods'], ['Single-Serve', 'Pods'], ['onces'], ['ounces.'], ['ounce'], ['grams', '(8.8', 'oz.)'], ['grams', '(12.3', 'ounces)'], ['grams', '(8.8', 'oz)'], ['pods'], ['64', 'pods'], ['T', 'Discs'], ['ouncues'], ['oz.'], ['g.', '(8.8', 'oz.)'], ['oz.', '(incl.', 'express', 'shipping', 'fr

In [119]:
df.drop(bad_units, inplace=True)

In [121]:
# remove indexes with grams

units = []
ounces = 0
bad_units = []
for i, price in df.Price.items():
    unit = ''.join(price.split()[1:])
    if 'g' in unit.split()[0]:
        bad_units.append(i)
    elif price.split()[1:] not in units:
        units.append(price.split()[1:])
print(units)
print(ounces)
print(bad_units)

[['ounces'], ['ounces;', '$50.00/5', 'pounds'], ['7', 'ounces'], ['ounces;', 'limited', 'availability'], ['ounces*'], ['ounces', 'online;', '$12.99/ounces', 'in', 'stores'], ['NTD/16', 'ounces'], ['ounces', '(see', 'notes)'], ['/', '12', 'ounces'], ['/', '8', 'ounces'], ['ounces.'], ['oz.'], ['/', '16', 'oz.'], ['/', '12', 'oz.'], ['/12', 'oz.'], ['/', '10', 'oz.']]
0
[8, 159, 160, 1558, 2965, 2981, 2989, 2990, 3012, 3040, 3041, 3064, 3124, 3125, 3128, 3161, 3164, 3195, 3242, 3248, 3270, 3284, 3285, 3286, 3288, 3400, 3401, 3403, 3405]


In [122]:
df.drop(bad_units, inplace=True)

In [137]:
# This one had NTD or something in the price
df.drop(1312, inplace=True)

In [150]:
import re

prices_per_oz = []
for i, price in df.Price.items():
    price = price.replace(' ', '')
    result = re.findall(r'([\.\d]+\/\d*)', price)
    prices = result[0].split('/')
    price_per_oz = float(prices[0]) / float(prices[1])
    prices_per_oz.append(round(price_per_oz, 2))
print(prices_per_oz)
        


[1.92, 1.96, 1.38, 1.35, 1.52, 6.24, 1.33, 1.49, 3.0, 1.12, 1.67, 7.14, 4.99, 4.99, 1.75, 1.66, 1.25, 1.0, 1.46, 1.5, 1.88, 1.62, 1.65, 1.58, 1.0, 3.75, 1.48, 1.35, 1.25, 1.5, 1.17, 0.91, 1.25, 1.17, 1.5, 1.17, 0.94, 0.44, 0.42, 0.42, 0.28, 0.87, 7.87, 1.58, 1.92, 1.5, 2.49, 1.14, 4.99, 1.65, 4.99, 1.33, 1.4, 1.35, 1.5, 1.5, 1.5, 1.56, 1.58, 1.25, 1.5, 1.33, 1.91, 1.12, 1.66, 1.75, 1.66, 1.75, 1.5, 1.41, 1.41, 11.66, 1.75, 1.83, 1.5, 1.75, 2.04, 1.43, 1.6, 1.41, 31.25, 7.5, 1.67, 1.5, 1.5, 1.67, 5.0, 2.0, 1.83, 1.5, 1.67, 1.5, 1.33, 1.69, 1.25, 1.67, 1.5, 1.21, 4.99, 2.49, 1.62, 7.5, 9.16, 7.5, 10.0, 6.25, 1.58, 1.43, 1.0, 1.0, 1.56, 9.99, 1.29, 1.83, 2.46, 1.35, 2.19, 2.08, 2.81, 1.75, 1.92, 1.58, 6.44, 1.5, 1.92, 1.42, 1.58, 6.0, 4.38, 12.49, 2.37, 3.75, 25.75, 1.37, 3.33, 1.25, 1.77, 2.5, 1.25, 1.25, 1.25, 1.5, 1.75, 1.32, 1.33, 1.25, 1.62, 2.08, 1.67, 5.0, 7.5, 1.58, 1.32, 1.66, 4.99, 10.5, 3.1, 2.9, 7.99, 6.12, 5.0, 3.0, 4.0, 6.25, 1.33, 3.5, 1.69, 2.82, 1.08, 1.54, 1.58, 1.35, 1.

In [224]:
print(type(prices_per_oz))

<class 'list'>


In [151]:
df['Prices Per Oz'] = prices_per_oz

In [186]:
df.head()

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line,Prices Per Oz
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...,1.92
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...,1.96
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...,1.38
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...,1.35
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...,1.52


In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2195 entries, 0 to 3689
Data columns (total 16 columns):
Coffee Name         2195 non-null object
Roaster Name        2195 non-null object
Roaster Location    2195 non-null object
Coffee Origin       2195 non-null object
Rating              2195 non-null object
Roast Level         2195 non-null object
Review Date         2195 non-null object
Price               2195 non-null object
Agtron              2195 non-null object
Flavor              2195 non-null object
Body                2195 non-null object
Aroma               2195 non-null object
Aftertaste          2195 non-null float64
Acidity             2195 non-null float64
Bottom Line         2189 non-null object
Prices Per Oz       2195 non-null float64
dtypes: float64(3), object(13)
memory usage: 291.5+ KB


In [ ]:
# price = df.iloc[2]['Price']
# print(price)

In [ ]:
# import re
# rePrice= re.findall('\d*\.?\d+',price)
# rePrice

In [ ]:
# df.Price.value_counts()
# Pound sign option-3-#

# df['temp1'] = df['Price'].str.contains('gram','g')

In [ ]:
# df['temp1'] = df['Price'].str.contains('NT')


In [ ]:
# df['temp1'] = df['Price'].str.contains('KRW')
# df['temp1'] = df['Price'].str.contains('RMB')
# df['temp1'] = df['Price'].str.contains('pods')
# df['temp1'] = df['Price'].str.contains('£')
# df['temp1'] = df['Price'].str.contains('NTD')
# df['temp1'] = df['Price'].str.contains('CAD')

In [ ]:
# df.temp1.value_counts()

In [69]:
# Attempted to write a function

# def currency_conversion(df):
#     if df['Price'].str.contains('ounces','oz.'):
#         price = df['Price']
#         rePrice = re.findall('\d*\.?\d+', price)
#         price_per_pound =  (rePrice[0] / (rePrice[1] / 16))
#         return price_per_pound
    

In [189]:
# df['New'] = df.apply(currency_conversion, axis=1)

## Split Agtron score into whole bean and ground

In [209]:
# agtron score split
df.info()
df.Agtron

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2195 entries, 0 to 3689
Data columns (total 16 columns):
Coffee Name         2195 non-null object
Roaster Name        2195 non-null object
Roaster Location    2195 non-null object
Coffee Origin       2195 non-null object
Rating              2195 non-null object
Roast Level         2195 non-null object
Review Date         2195 non-null object
Price               2195 non-null object
Agtron              2195 non-null object
Flavor              2195 non-null object
Body                2195 non-null object
Aroma               2195 non-null object
Aftertaste          2195 non-null float64
Acidity             2195 non-null float64
Bottom Line         2189 non-null object
Prices Per Oz       2195 non-null float64
dtypes: float64(3), object(13)
memory usage: 371.5+ KB


0       58/76
1       60/74
3       58/76
5       56/74
6       58/76
7       60/78
9       58/78
11      56/72
13      52/68
14      52/68
18      54/70
21      56/70
23      57/73
24      59/73
25      58/82
26      55/73
27      47/67
28      56/72
29      58/77
30      58/80
31      52/68
32      54/74
36      58/78
37      56/74
38      52/68
40      54/72
41      59/76
42      54/71
46      58/74
47      54/72
        ...  
3407    50/68
3408    48/53
3409    29/52
3410    60/74
3413    50/65
3414    53/80
3415    40/65
3416    49/62
3417    51/72
3418    58/73
3419    46/60
3420    58/85
3421    74/99
3425    59/82
3426    44/63
3427     0/45
3428    55/68
3431    64/95
3432    43/51
3433    43/54
3434    62/77
3435    39/70
3437    41/57
3438    60/86
3439    59/94
3440    51/84
3441    66/99
3442    52/90
3443    54/71
3689    63/86
Name: Agtron, Length: 2195, dtype: object

In [216]:
agt = df.loc[3427]['Agtron']
# price = df['Price']
reA = re.findall('\d*\.?\d+', agt)
print(reA[0])
print(reA[1])
print(type(reA[0]))

0
45
<class 'str'>


In [217]:
Agt1 = reA[0]
Agt2 = reA[1]

In [220]:
# Split Agtron score into 2 

agtron1 = []
agtron2 = []

for i, score in df.Agtron.items():
#     price = price.replace(' ', '')
    result = re.findall(r'\d*\.?\d+', score)
    result1 = result[0]
    result2 = result[1]

    agtron1.append(result1)
    agtron2.append(result2)

print(agtron1)

['58', '60', '58', '56', '58', '60', '58', '56', '52', '52', '54', '56', '57', '59', '58', '55', '47', '56', '58', '58', '52', '54', '58', '56', '52', '54', '59', '54', '58', '54', '47', '52', '52', '42', '53', '54', '50', '44', '40', '46', '41', '31', '53', '56', '58', '60', '58', '47', '56', '52', '58', '50', '58', '54', '50', '47', '45', '47', '51', '53', '48', '48', '60', '36', '58', '59', '54', '58', '54', '59', '60', '59', '56', '58', '56', '58', '51', '52', '56', '55', '56', '52', '54', '55', '64', '60', '52', '56', '58', '63', '58', '54', '54', '52', '47', '59', '58', '69', '54', '50', '54', '54', '57', '51', '53', '60', '59', '52', '53', '58', '54', '52', '55', '54', '47', '54', '55', '55', '53', '50', '47', '54', '52', '58', '54', '52', '53', '60', '56', '54', '52', '56', '57', '55', '53', '56', '54', '54', '57', '52', '58', '60', '58', '52', '58', '60', '57', '56', '60', '50', '57', '55', '58', '63', '58', '51', '54', '50', '56', '56', '58', '52', '57', '52', '52', '49', '53

In [223]:
print(type(agtron2))

<class 'list'>


In [225]:
df['Agtron Whole'] = agtron1

In [226]:
df['Agtron Ground'] = agtron2

In [227]:
df.head()

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line,Prices Per Oz,Agtron Whole,Agtron Ground
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...,1.92,58,76
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...,1.96,60,74
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...,1.38,58,76
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...,1.35,56,74
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...,1.52,58,76


## Countries

In [704]:
# !pip install pycountry

In [235]:
# import pycountry

In [251]:
# deleted Czech Republic, Syrian Arab Republic, Central African Republic, South Africa, Equatorial Guinea, 
# Gineau and Guinea-bissau, American Somoa

In [742]:
Country = [
    ('US', 'United States'),
    ('AF', 'Afghanistan'),
    ('AL', 'Albania'),
    ('DZ', 'Algeria'),
    ('AD', 'Andorra'),
    ('AO', 'Angola'),
    ('AI', 'Anguilla'),
    ('AQ', 'Antarctica'),
    ('AG', 'Antigua And Barbuda'),
    ('AR', 'Argentina'),
    ('AM', 'Armenia'),
    ('AW', 'Aruba'),
    ('AU', 'Australia'),
    ('AT', 'Austria'),
    ('AZ', 'Azerbaijan'),
    ('BS', 'Bahamas'),
    ('BH', 'Bahrain'),
    ('BD', 'Bangladesh'),
    ('BB', 'Barbados'),
    ('BY', 'Belarus'),
    ('BE', 'Belgium'),
    ('BZ', 'Belize'),
    ('BJ', 'Benin'),
    ('BM', 'Bermuda'),
    ('BT', 'Bhutan'),
    ('BO', 'Bolivia'),
    ('BA', 'Bosnia And Herzegowina'),
    ('BW', 'Botswana'),
    ('BV', 'Bouvet Island'),
    ('BR', 'Brazil'),
    ('BN', 'Brunei Darussalam'),
    ('BG', 'Bulgaria'),
    ('BF', 'Burkina Faso'),
    ('BI', 'Burundi'),
    ('KH', 'Cambodia'),
    ('CM', 'Cameroon'),
    ('CA', 'Canada'),
    ('CV', 'Cape Verde'),
    ('KY', 'Cayman Islands'),
    ('TD', 'Chad'),
    ('CL', 'Chile'),
    ('CN', 'China'),
    ('CX', 'Christmas Island'),
    ('CC', 'Cocos Islands'),
    ('CO', 'Colombia'),
    ('KM', 'Comoros'),
    ('CG', 'Congo'),
    ('CK', 'Cook Islands'),
    ('CR', 'Costa Rica'),
    ('CI', 'Cote D`ivoire'),
    ('HR', 'Croatia'),
    ('CU', 'Cuba'),
    ('CY', 'Cyprus'),
    ('DK', 'Denmark'),
    ('DJ', 'Djibouti'),
    ('DM', 'Dominica'),
    ('DO', 'Dominican Republic'),
    ('TP', 'East Timor'),
    ('EC', 'Ecuador'),
    ('EG', 'Egypt'),
    ('SV', 'El Salvador'),
    ('ER', 'Eritrea'),
    ('EE', 'Estonia'),
    ('ET', 'Ethiopia'),
    ('FK', 'Falkland Islands (Malvinas)'),
    ('FO', 'Faroe Islands'),
    ('FJ', 'Fiji'),
    ('FI', 'Finland'),
    ('FR', 'France'),
    ('GF', 'French Guiana'),
    ('PF', 'French Polynesia'),
    ('TF', 'French S. Territories'),
    ('GA', 'Gabon'),
    ('GM', 'Gambia'),
    ('GE', 'Georgia'),
    ('DE', 'Germany'),
    ('GH', 'Ghana'),
    ('GI', 'Gibraltar'),
    ('GR', 'Greece'),
    ('GL', 'Greenland'),
    ('GD', 'Grenada'),
    ('GP', 'Guadeloupe'),
    ('GU', 'Guam'),
    ('GT', 'Guatemala'),
    ('GY', 'Guyana'),
    ('HT', 'Haiti'),
    ('HN', 'Honduras'),
    ('HK', 'Hong Kong'),
    ('HU', 'Hungary'),
    ('IS', 'Iceland'),
    ('IN', 'India'),
    ('ID', 'Indonesia'),
    ('IR', 'Iran'),
    ('IQ', 'Iraq'),
    ('IE', 'Ireland'),
    ('IL', 'Israel'),
    ('IT', 'Italy'),
    ('JM', 'Jamaica'),
    ('JP', 'Japan'),
    ('JO', 'Jordan'),
    ('KZ', 'Kazakhstan'),
    ('KE', 'Kenya'),
    ('KI', 'Kiribati'),
    ('KP', 'Korea (North)'),
    ('KR', 'Korea (South)'),
    ('KW', 'Kuwait'),
    ('KG', 'Kyrgyzstan'),
    ('LA', 'Laos'),
    ('LV', 'Latvia'),
    ('LB', 'Lebanon'),
    ('LS', 'Lesotho'),
    ('LR', 'Liberia'),
    ('LY', 'Libya'),
    ('LI', 'Liechtenstein'),
    ('LT', 'Lithuania'),
    ('LU', 'Luxembourg'),
    ('MO', 'Macau'),
    ('MK', 'Macedonia'),
    ('MG', 'Madagascar'),
    ('MW', 'Malawi'),
    ('MY', 'Malaysia'),
    ('MV', 'Maldives'),
    ('ML', 'Mali'),
    ('MT', 'Malta'),
    ('MH', 'Marshall Islands'),
    ('MQ', 'Martinique'),
    ('MR', 'Mauritania'),
    ('MU', 'Mauritius'),
    ('YT', 'Mayotte'),
    ('MX', 'Mexico'),
    ('FM', 'Micronesia'),
    ('MD', 'Moldova'),
    ('MC', 'Monaco'),
    ('MN', 'Mongolia'),
    ('MS', 'Montserrat'),
    ('MA', 'Morocco'),
    ('MZ', 'Mozambique'),
    ('MM', 'Myanmar'),
    ('NA', 'Namibia'),
    ('NR', 'Nauru'),
    ('NP', 'Nepal'),
    ('NL', 'Netherlands'),
    ('AN', 'Netherlands Antilles'),
    ('NC', 'New Caledonia'),
    ('NZ', 'New Zealand'),
    ('NI', 'Nicaragua'),
    ('NE', 'Niger'),
    ('NG', 'Nigeria'),
    ('NU', 'Niue'),
    ('NF', 'Norfolk Island'),
    ('MP', 'Northern Mariana Islands'),
    ('NO', 'Norway'),
    ('OM', 'Oman'),
    ('PK', 'Pakistan'),
    ('PW', 'Palau'),
    ('PA', 'Panama'),
    ('PG', 'Papua New Guinea'),
    ('PY', 'Paraguay'),
    ('PE', 'Peru'),
    ('PH', 'Philippines'),
    ('PN', 'Pitcairn'),
    ('PL', 'Poland'),
    ('PT', 'Portugal'),
    ('PR', 'Puerto Rico'),
    ('QA', 'Qatar'),
    ('RE', 'Reunion'),
    ('RO', 'Romania'),
    ('RU', 'Russian Federation'),
    ('RW', 'Rwanda'),
    ('KN', 'Saint Kitts And Nevis'),
    ('LC', 'Saint Lucia'),
    ('VC', 'St Vincent/Grenadines'),
    ('WS', 'Samoa'),
    ('SM', 'San Marino'),
    ('ST', 'Sao Tome'),
    ('SA', 'Saudi Arabia'),
    ('SN', 'Senegal'),
    ('SC', 'Seychelles'),
    ('SL', 'Sierra Leone'),
    ('SG', 'Singapore'),
    ('SK', 'Slovakia'),
    ('SI', 'Slovenia'),
    ('SB', 'Solomon Islands'),
    ('SO', 'Somalia'),
    ('ES', 'Spain'),
    ('LK', 'Sri Lanka'),
    ('SH', 'St. Helena'),
    ('PM', 'St.Pierre'),
    ('SD', 'Sudan'),
    ('SR', 'Suriname'),
    ('SZ', 'Swaziland'),
    ('SE', 'Sweden'),
    ('CH', 'Switzerland'),
    ('TW', 'Taiwan'),
    ('TJ', 'Tajikistan'),
    ('TZ', 'Tanzania'),
    ('TH', 'Thailand'),
    ('TG', 'Togo'),
    ('TK', 'Tokelau'),
    ('TO', 'Tonga'),
    ('TT', 'Trinidad And Tobago'),
    ('TN', 'Tunisia'),
    ('TR', 'Turkey'),
    ('TM', 'Turkmenistan'),
    ('TV', 'Tuvalu'),
    ('UG', 'Uganda'),
    ('UA', 'Ukraine'),
    ('AE', 'United Arab Emirates'),
    ('UK', 'United Kingdom'),
    ('UY', 'Uruguay'),
    ('UZ', 'Uzbekistan'),
    ('VU', 'Vanuatu'),
    ('VA', 'Vatican City State'),
    ('VE', 'Venezuela'),
    ('VN', 'Viet Nam'),
    ('VG', 'Virgin Islands (British)'),
    ('VI', 'Virgin Islands (U.S.)'),
    ('EH', 'Western Sahara'),
    ('YE', 'Yemen'),
    ('YU', 'Yugoslavia'),
    ('ZR', 'Zaire'),
    ('ZM', 'Zambia'),
    ('ZW', 'Zimbabwe')
]

In [743]:
print(len(Country))
temp = Country[0][1]
print(temp)

223
United States


In [744]:
cleaned_countries = []

for p in Country:
#     print('{}'.format(p[1]))   
    temp = p[1]
    cleaned_countries.append(temp)
    
print(cleaned_countries)

['United States', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia And Herzegowina', 'Botswana', 'Bouvet Island', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Cote D`ivoire', 'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French S. Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', '

In [810]:
result = [i for i in cleaned_countries if 'Republic' in i]
print(result)

In [745]:
# df['Coffee Origin'].head()

In [517]:
# df['Coffee Origin'].value_counts()

In [746]:
# Number of countries with ; in name

test = df[df['Coffee Origin'].str.contains(';')]
len(test)
# print(test)

0

In [713]:
# Change all countries with a ; to World Blend

for i, country in df['Coffee Origin'].items():
    if ";" in country:
        df.at[i, 'Coffee Origin'] = 'World Blend'
#         print(df.at[i, 'Coffee Origin'])

In [747]:
# want to see results of countries
test = df[df['Coffee Origin'].str.contains('World')]
len(test)
# print(test)

90

In [716]:
# Fix Hawaii in dataframe

# df['a'] = df['a'].str.replace('Hi', 'hi')
# print(df['Coffee Origin'].where(df['Coffee Origin'] == 'Hawaii').count())
# df['Coffee Origin'].nunique
# print(len([df['Coffee Origin'] == 'Hawaii']))

print(df['Coffee Origin'].str.count("Hawaii").sum())
print(df['Coffee Origin'].str.count("Hawai'i").sum())
print("Hawaii w split ", df['Coffee Origin'].str.contains("Hawai'i").sum())
print("Hawaii w no split ", df['Coffee Origin'].str.contains("Hawaii").sum())

89
0
Hawaii w split  0
Hawaii w no split  89


In [717]:
# df['Coffee Origin'] = df['Coffee Origin'].astype(str)
# type(df['Coffee Origin'])

In [718]:
# this had to be cut/pasted from the df Hawai’i

df['Coffee Origin'] = df['Coffee Origin'].str.replace('Hawai’i','Hawaii')
df.head(10)

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line,Prices Per Oz,Agtron Whole,Agtron Ground
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...,1.92,58,76
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...,1.96,60,74
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...,1.38,58,76
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...,1.35,56,74
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...,1.52,58,76
7,Ka’u Yellow Caturra Champagne Natural,Paradise Roasters,"Minneapolis, Minnesota","Ka’u growing region, Big Island of Hawaii",93,Medium-Light,March 2020,$24.95/4 ounces,60/78,9,8,9,8.0,9.0,"A complex, nuanced Ka’u coffee processed with ...",6.24,60,78
9,Colombia San Pascual Natural,Magnolia Coffee,"Charlotte, North Carolina","Fredonia, Antioquia Department, Colombia",92,Medium-Light,March 2020,$16.00/12 ounces; $50.00/5 pounds,58/78,9,9,8,8.0,8.0,A sweetly fermenty and chocolaty Colombia cup ...,1.33,58,78
11,Birambo Village DR Congo,JBC Coffee Roasters,"Madison, Wisconsin","South Kivu Province, Democratic Republic of th...",91,Medium-Light,March 2020,$17.90/12 ounces,56/72,9,8,8,8.0,8.0,"A balanced, sweetly savory DR Congo cup with d...",1.49,56,72
13,KonAroma Direct Trade Kona,Jampot Poorrock Abbey,"Eagle Harbor, Michigan","Kona growing district, Big Island of Hawaii",92,Medium,March 2020,$24.00/8 ounces,52/68,9,8,9,8.0,8.0,"A pretty, richly sweet Kona cup that leads wit...",3.00,52,68
14,Costa Rica Tarrazu Hacienda La Pradera,Jampot Poorrock Abbey,"Eagle Harbor, Michigan","Tarrazu, Costa Rica",91,Medium,March 2020,$18.00/16 ounces,52/68,9,8,8,8.0,8.0,"A balanced washed process Costa Rica cup, live...",1.12,52,68


In [719]:
print("Hawaii w split ", df['Coffee Origin'].str.contains("Hawai’i").sum())

Hawaii w split  0


In [720]:
print("Hawaii w no split ", df['Coffee Origin'].str.contains("Hawaii").sum())

Hawaii w no split  89


In [721]:
# Not disclosed/Not Disclosed 

print("ND ", df['Coffee Origin'].str.contains("Disclosed").sum())
print("Nd", df['Coffee Origin'].str.contains("Not disclosed").sum())

ND  0
Nd 0


In [722]:
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Not disclosed','undisclosed')

# type in the first try
# df['Coffee Origin'] = df['Coffee Origin'].str.replace('undislosed','undisclosed')

In [723]:
print("Result", df['Coffee Origin'].str.contains('Guinea').sum())

Result 22


In [724]:
df.head()

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line,Prices Per Oz,Agtron Whole,Agtron Ground
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...,1.92,58,76
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...,1.96,60,74
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...,1.38,58,76
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...,1.35,56,74
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...,1.52,58,76


In [748]:
# these are the outlier locations added to the countries list
# cleaned_countries.append('South America')
# cleaned_countries.append('Central America')

cleaned_countries.append('Hawaii')
cleaned_countries.append('California')
cleaned_countries.append('Africa')
cleaned_countries.append('undisclosed')
cleaned_countries.append('World Blend')

In [726]:
# these need to be replaced

df['Coffee Origin'] = df['Coffee Origin'].str.replace('Sumatra','Indonesia')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Saint Helena Island','Africa')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Java','Indonesia')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Minas Gerais State','Brazil')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Latin America','Peru')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Sandy Bay Valley, Africa','Africa')


In [766]:
# these need to be fixed to: no Central America, South America or mix

df['Coffee Origin'] = df['Coffee Origin'].str.replace('South America','Colombia')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Central America','Costa Rica')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Central and South America','World Blend')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Various South American origins','Colombia')

In [821]:
print("Result", df['Coffee Origin'].str.contains('Mur').sum())

Result 15


In [822]:

count = 0
for i, country in df['Coffee Origin'].items():
    if "Mur" in country:
        count += 1
        print(df.at[i, 'Coffee Origin'])          

Murutra, Kayanza Province, Burundi
Murang'a County, central Kenya
Muranga District, Central Province, Kenya
Murang'a County, central Kenya
Muranga County, south-central KenyaMur
Murang’a Foothills, Aberdare Ridge, Central Province, Kenya
Maragua, Muranga County, Kenya
Murang’a County, central Kenya
Muramvya Province, Burundi.
Murang’a County, central Kenya.
Murang’a County, Central Kenya.
Murang’a County, Central Kenya.
Murang’a County, central Kenya.
Muranga District, central Kenya.
Murang'a, south-central Kenya


In [848]:
# More to fix
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Southern Province,Rwanda','Rwanda')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('Democratic Republic of the Congo','Congo')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('ColombiaNar','Colombia')
df['Coffee Origin'] = df['Coffee Origin'].str.replace('KenyaMur','Kenya')


In [849]:
# finished list
cleaned_countries

['United States',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua And Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia And Herzegowina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos Islands',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Cote D`ivoire',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Malvinas)',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia

In [850]:
count = 0
for i in cleaned_countries:
    if len(i.split()) > 1:
        print(i, " has ", df['Coffee Origin'].str.contains(i).sum(),'occurances')

United States  has  0 occurances
Antigua And Barbuda  has  0 occurances
Bosnia And Herzegowina  has  0 occurances
Bouvet Island  has  0 occurances
Brunei Darussalam  has  0 occurances
Burkina Faso  has  0 occurances
Cape Verde  has  0 occurances
Cayman Islands  has  0 occurances
Christmas Island  has  0 occurances
Cocos Islands  has  0 occurances
Cook Islands  has  0 occurances
Costa Rica  has  119 occurances
Cote D`ivoire  has  0 occurances
Dominican Republic  has  7 occurances
East Timor  has  0 occurances
El Salvador  has  87 occurances
Falkland Islands (Malvinas)  has  0 occurances
Faroe Islands  has  0 occurances
French Guiana  has  0 occurances
French Polynesia  has  0 occurances
French S. Territories  has  0 occurances
Hong Kong  has  0 occurances
Korea (North)  has  0 occurances
Korea (South)  has  0 occurances
Marshall Islands  has  0 occurances
Netherlands Antilles  has  0 occurances
New Caledonia  has  0 occurances
New Zealand  has  0 occurances
Norfolk Island  has  0 occura

In [851]:
# print("Result", df['Coffee Origin'].str.contains('Sri Lanka').sum())

In [852]:
# text = df['Coffee Origin'][0]
# print(text)

In [767]:
# Checking for countries to be in the list of cleaned_countries

results = []
fix_location = []

for i, country in df['Coffee Origin'].items():
        test_str = country
#         print(test_str)
#         if country in cleaned_countries:
#             print(cleaned_countries)
        result = any(ele in test_str for ele in cleaned_countries)
        results.append(result)
        if result == False:
            fix_location.append(i)
    
# for country in cleaned_countries:
#     if country in text:
#         print(country)

In [765]:
df['Coffee Origin'][1598]

'Various South American origins.'

In [771]:
fix_location

[]

In [824]:
countries = df['Coffee Origin']

In [825]:
countries[-5:]

3440       Kona growing district, "Big Island" of Hawaii.
3441    Ka'u growing district, southwestern corner of ...
3442    Ka'u growing district, southwestern corner of ...
3443                                   Dominican Republic
3689                                          Costa Rica.
Name: Coffee Origin, dtype: object

In [867]:
# This splits the name on white space and reads the last word
# Figured out the number of 2 or 3 named countries and re-addded them

country_results = []

for country in countries:
        country_split = country.split(' ')
#         print(country_split[-1])        
        last = country_split[-1].strip('.')
#         print(last)
        if last == "Blend":
            last = "World Blend"
        elif last == "Rica":
            last = "Costa Rica"
        elif last == "Republic":
            last = "Dominican Republic"
        elif last == "Salvador":
            last = "El Salvador"
        elif last == "Guinea":
            last = "Papua New Guinea"
        elif last == "Rico":
            last = "Puerto Rico"
        country_results.append(last)


In [868]:
country_results

['Costa Rica',
 'Colombia',
 'Colombia',
 'Rwanda',
 'Burundi',
 'Hawaii',
 'Colombia',
 'Congo',
 'Hawaii',
 'Costa Rica',
 'Panama',
 'Hawaii',
 'Hawaii',
 'Hawaii',
 'Ethiopia',
 'Hawaii',
 'Peru',
 'Tanzania',
 'Ethiopia',
 'Ethiopia',
 'Kenya',
 'Ethiopia',
 'Ethiopia',
 'Kenya',
 'Kenya',
 'Panama',
 'Colombia',
 'Peru',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Colombia',
 'Kenya',
 'Guatemala',
 'Honduras',
 'Ethiopia',
 'Ethiopia',
 'Guatemala',
 'Colombia',
 'Colombia',
 'Guatemala',
 'Indonesia',
 'Hawaii',
 'Guatemala',
 'El Salvador',
 'Ethiopia',
 'Madagascar',
 'Costa Rica',
 'Panama',
 'Ethiopia',
 'Hawaii',
 'Costa Rica',
 'Ethiopia',
 'Colombia',
 'Rwanda',
 'Guatemala',
 'World Blend',
 'World Blend',
 'World Blend',
 'Peru',
 'World Blend',
 'World Blend',
 'World Blend',
 'World Blend',
 'Ethiopia',
 'Kenya',
 'Kenya',
 'Guatemala',
 'Costa Rica',
 'Guatemala',
 'El Salvador',
 'Panama',
 'Ethiopia',
 'Panama',
 'Ethiopia',
 'Kenya',
 'Kenya',
 'Indonesia',
 'Kenya'

In [829]:
# # This only works if the country name is one word, not if it is part of a long name

# good_count = 0
# bad_count = 0

# for i, country in df['Coffee Origin'].items():
#     if country in cleaned_countries:
#         good_count += 1
#     else: 
#         bad_count+= 1
# #         df_copy.at[i, 'Coffee Origin'] = 'World Blend'
# #         print(df_copy.at[i, 'Coffee Origin'])

In [828]:
print(good_count)
print(bad_count)

179
2016


In [869]:
df['Coffee Country'] = country_results

In [871]:
df.head(10)

,Coffee Name,Roaster Name,Roaster Location,Coffee Origin,Rating,Roast Level,Review Date,Price,Agtron,Flavor,Body,Aroma,Aftertaste,Acidity,Bottom Line,Prices Per Oz,Agtron Whole,Agtron Ground,Coffee Country
0,Costa Rica Luis Campos Anaerobic,Revel Coffee,"Billings, Montana","Tarrazu, Costa Rica",94,Medium-Light,April 2020,$23.00/12 ounces,58/76,9,9,9,8.0,9.0,Enjoying this Costa Rica evokes the pleasures ...,1.92,58,76,Costa Rica
1,Colombia Granja La Esperanza Tres Dragones,PT's Coffee Roasting Co.,"Topeka, Kansas","Valle del Cauca growing region, Colombia",94,Medium-Light,April 2020,$23.50/12 ounces,60/74,9,9,9,8.0,9.0,A bombshell of a coffee with enough personalit...,1.96,60,74,Colombia
3,Colombia Finca La Loma Microlot,BeanFruit Coffee Co.,"Jackson, Mississippi","Huila, Colombia",94,Medium-Light,March 2020,$16.50/12 ounces,58/76,9,9,9,8.0,9.0,An exceptional Colombia cup with sweet and tar...,1.38,58,76,Colombia
5,Dukunde Kawa Rwanda,JBC Coffee Roasters,"Madison, Wisconsin","Musasa, Rwanda",93,Medium-Light,March 2020,$16.25/12 ounces,56/74,9,9,9,8.0,8.0,A deep yet delicate cup redolent with crisp fr...,1.35,56,74,Rwanda
6,Buzira Burundi Natural,JBC Coffee Roasters,"Madison, Wisconsin","Murutra, Kayanza Province, Burundi",92,Medium-Light,March 2020,$18.20/12 ounces,58/76,9,8,9,8.0,8.0,An exceptionally cleanly expressed natural-pro...,1.52,58,76,Burundi
7,Ka’u Yellow Caturra Champagne Natural,Paradise Roasters,"Minneapolis, Minnesota","Ka’u growing region, Big Island of Hawaii",93,Medium-Light,March 2020,$24.95/4 ounces,60/78,9,8,9,8.0,9.0,"A complex, nuanced Ka’u coffee processed with ...",6.24,60,78,Hawaii
9,Colombia San Pascual Natural,Magnolia Coffee,"Charlotte, North Carolina","Fredonia, Antioquia Department, Colombia",92,Medium-Light,March 2020,$16.00/12 ounces; $50.00/5 pounds,58/78,9,9,8,8.0,8.0,A sweetly fermenty and chocolaty Colombia cup ...,1.33,58,78,Colombia
11,Birambo Village DR Congo,JBC Coffee Roasters,"Madison, Wisconsin","South Kivu Province, Congo",91,Medium-Light,March 2020,$17.90/12 ounces,56/72,9,8,8,8.0,8.0,"A balanced, sweetly savory DR Congo cup with d...",1.49,56,72,Congo
13,KonAroma Direct Trade Kona,Jampot Poorrock Abbey,"Eagle Harbor, Michigan","Kona growing district, Big Island of Hawaii",92,Medium,March 2020,$24.00/8 ounces,52/68,9,8,9,8.0,8.0,"A pretty, richly sweet Kona cup that leads wit...",3.00,52,68,Hawaii
14,Costa Rica Tarrazu Hacienda La Pradera,Jampot Poorrock Abbey,"Eagle Harbor, Michigan","Tarrazu, Costa Rica",91,Medium,March 2020,$18.00/16 ounces,52/68,9,8,8,8.0,8.0,"A balanced washed process Costa Rica cup, live...",1.12,52,68,Costa Rica


In [879]:
df['Coffee Country'].nunique()

44

# Fix data types

In [889]:
df.dtypes

Coffee Name          object
Roaster Name         object
Roaster Location     object
Coffee Origin        object
Rating               object
Roast Level          object
Review Date          object
Price                object
Agtron               object
Flavor               object
Body                 object
Aroma                object
Aftertaste          float64
Acidity             float64
Bottom Line          object
Prices Per Oz       float64
Agtron Whole         object
Agtron Ground        object
Coffee Country       object
dtype: object

In [ ]:
# these should be ints: rating, flavor, body, aroma, aftertaste, agtron whole, agtron ground
# check acidity for nan

# df['Acidity '] = df['Acidity '].apply(pd.to_numeric, errors='coerce')

In [882]:
# 0 
df.Acidity.isna().sum()

0

In [892]:
df.Rating = df.Rating.apply(pd.to_numeric)

In [898]:
df.Flavor = df.Flavor.astype(int)

In [901]:
df.Body = df.Body.astype(int)

In [902]:
df.Aroma = df.Aroma.astype(int)

In [903]:
df.Aftertaste = df.Aftertaste.astype(int)

In [904]:
df['Agtron Whole'] = df['Agtron Whole'].astype(int)

In [905]:
df['Agtron Ground'] = df['Agtron Ground'].astype(int)

In [906]:
df.dtypes

Coffee Name          object
Roaster Name         object
Roaster Location     object
Coffee Origin        object
Rating                int64
Roast Level          object
Review Date          object
Price                object
Agtron               object
Flavor                int64
Body                  int64
Aroma                 int64
Aftertaste            int64
Acidity             float64
Bottom Line          object
Prices Per Oz       float64
Agtron Whole          int64
Agtron Ground         int64
Coffee Country       object
dtype: object

In [908]:
# 6 missing Bottom Line reviews
# If I decide to do a sentiment analysis, they can be deleted otherwise I'll keep them
df.isna().sum()

Coffee Name         0
Roaster Name        0
Roaster Location    0
Coffee Origin       0
Rating              0
Roast Level         0
Review Date         0
Price               0
Agtron              0
Flavor              0
Body                0
Aroma               0
Aftertaste          0
Acidity             0
Bottom Line         6
Prices Per Oz       0
Agtron Whole        0
Agtron Ground       0
Coffee Country      0
dtype: int64

In [909]:
df.to_csv('cleaned_coffee_reviews.csv')

# Code Holding

In [696]:
# change the countries with ; in the country names to World Blend

# df_copy = df.copy()

In [710]:
# df_copy.head(1)

In [711]:
# # want to see results of countries
# test = df_copy[df_copy['Coffee Origin'].str.contains(';')]
# len(test)
# # print(test)

In [709]:
# https://stackoverflow.com/questions/26657378/how-to-modify-a-value-in-one-cell-of-a-pandas-data-frame
#         df.at[this_date, 'column3'] = val

# count = 0
# for i, country in df_copy['Coffee Origin'].items():
#     if ";" in country:
#         count += 1
#         df_copy.at[i, 'Coffee Origin'] = 'World Blend'
#         print(df_copy.at[i, 'Coffee Origin'])

In [701]:
# # want to see results of countries
# test = df_copy[df_copy['Coffee Origin'].str.contains(';')]
# len(test)
# # print(test)

0

In [703]:
# df_copy['Coffee Origin'].value_counts()

In [354]:
# for i,row in titanic.iterrows():
#     titanic.at[i, "Name"] = row.Name.replace("'", "''")
#     titanic.at[i, "Ticket"] = row.Ticket.replace("'", "''")

In [702]:
# Coffee origin
# if not disclosed or a mix, then set it to unknown?

# countries =['Ethiopia','Panama','Kenya','Columbia','Guatmemala','Indonesia','Costa Rica','Hawaii','Honduras',
#            'Haiti','Mexico','Tanzania','Yemen','Hawai"i', 'Myanmar','Nicaragua','Colombia; Kenya; Sumatra',
#            'El Salvador','Brazil','Burundi','Rwanda','Congo','Uganda','Not disclosed','Peru','Jamaica']



In [59]:
# df = df.drop(['New'], axis=1)

In [272]:
# df[df['Coffee Origin'].str.match("Mexico")]
# https://www.pythonprogramming.in/if-value-in-row-in-dataframe-contains-string-create-another-column-equal-to-string-in-pandas.html

# df['New'] = pd.np.where(df['Coffee Origin'].str.contains("Mexico"), "Mexico",
#             pd.np.where(df['Coffee Origin'].str.contains("Ethiopia"), "Ethiopia",
#             pd.np.where(df['Coffee Origin'].str.contains("Panama"), "Panama")))

In [ ]:
#         for i in country_split:
#         result = [x for x in cleaned_countries if last in x]
#         print(result)

In [ ]:
        
#         for i in cleaned_countries:
#             print(i)
#             if country.str.contains(cleaned_countries):
#                 print("yes")
#         result = cleaned_countries.contains(country)
#         print(result)
            
#         result = [i for i in cleaned_countries if substring in i] 

#         results.append(result)
    
#         result = pd.np.where(df.Occupation.str.contains("Chemist"), "Science")